In [ ]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)
import scipy.signal as signal

In [ ]:
subj_uuid = (subject.Subject & 'subject_nickname="IBL-T3"').fetch1('subject_uuid')
subj = subject.Subject & {'subject_uuid': subj_uuid}
key = {'subject_uuid': subj_uuid,
   'session_date': datetime.date(2019, 4, 22)}

In [ ]:
# get all trial of the day
trial_sets = (behavior.TrialSet & 
              (behavior.CompleteTrialSession & 
               'stim_on_times_status="Complete"')).proj(
    session_date='DATE(session_start_time)')
trials = behavior.TrialSet.Trial & (behavior.TrialSet * trial_sets & key)
rt_trials = trials.proj(rt='trial_response_time-trial_stim_on_time').fetch(as_dict=True)
rt_trials = pd.DataFrame(rt_trials)
rt_trials.index = rt_trials.index + 1
rt_rolled = rt_trials['rt'].rolling(window=10).median()
rt_rolled = rt_rolled.where((pd.notnull(rt_rolled)), None)


data = dict(
    x=rt_trials.index.tolist(),
    y=rt_trials['rt'].tolist(),
    name='data',
    type='scatter',
    mode='markers',
    marker=dict(
        color='lightgray'
    )
)

rolled = dict(
    x=rt_trials.index.tolist(),
    y=rt_rolled.values.tolist(),
    name='rolled data',
    type='scatter',
    marker=dict(
        color='black'
    )
)

data = [data, rolled]

layout = go.Layout(
    width=630,
    height=400,
    title=dict(
        text='Reaction time - trial number',
        x=0.26,
        y=0.85
    ),
    xaxis=dict(
        title='Trial number',
        showgrid=True
    ), 
    yaxis=dict(
        title='Reaction time (s)',
        type='log',
        range=np.log10([0.1, 100]).tolist(),
        dtick=np.log10([0.1, 1, 10, 100]).tolist()),
    template=dict(
        layout=dict(
           plot_bgcolor="white"
        ))
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [ ]:
f = open("reaction_time_trial_number.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()